# MQTT

- MQTT (Message Queue Telemetry Transport)
- 為M2M, IOT設計的協定，基於TCP socket，主要是想讓系統資源較低的設備也可以把資料上傳到Server。
- 目前已是ISO的一員:
    - 2016, MQTT is now an ISO standard (ISO/IEC 20922)
- 本次教學中的MQTT雲端服務為Eclipse所提供：iot.eclipse.org    
- 架構如下：
    - 有興趣可以參考此[文章](https://www.javacodegeeks.com/2016/10/mqtt-protocol-tutorial.html)進一步了解
![mqtt](https://www.javacodegeeks.com/wp-content/uploads/2016/10/mqtt_publisher_subscriber-1.png)


# 利用手機和你的IOT設備互動


----
## 圖文併茂的Android手機監控APP - Linear MQTT Dashboard

* 先在android play上安裝Linear MQTT Dashboard
* 打開APP後的操作流程如下圖：
    * Led燈：malo-iot/light
    * 溫度：malo-iot/T
    * 溼度：malo-iot/H
    
![linear-mqtt1](image/linear-mqtt.png)

In [1]:
%websocketconnect --password 1234 ws://192.168.43.245:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [8]:
# demo: push T/H data to MQTT Server

from umqtt.simple import MQTTClient
from machine import Pin
import dht
import ubinascii
import machine
import network
import time
import os

TOPIC_BASE = 'malo-iot'

def dht_get():
    T=None
    H=None
    try:
        d = dht.DHT11(Pin(5)) #D1

        d.measure()
        T = d.temperature()
        H = d.humidity()
    except Exception as e:
        print('dht_get error:', str(e))
    
    return T, H


# Default MQTT server to connect to
server = "iot.eclipse.org"
CLIENT_ID = ubinascii.hexlify(machine.unique_id()).decode('utf-8')
topic_t = TOPIC_BASE+'/T'
topic_h = TOPIC_BASE+'/H'

c = MQTTClient(CLIENT_ID, server)
c.connect()

tm_pub_th = time.ticks_ms()

time.sleep(1)
T, H = dht_get()
print("T=%s, H=%s" %(T, H))
c.publish(topic_t, str(T))
c.publish(topic_h, str(H))

c.disconnect()

T=30, H=88


### 有資訊push上來後的畫面

![linear-mqtt1](image/linear-mqtt2.png)

In [10]:
# send T/H sensor data to MQTT cloud platform
# control light with MQTT

from umqtt.simple import MQTTClient
from machine import Pin
import dht
import ubinascii
import machine
import network
import time
import os

# ESP8266 ESP-12 modules have blue, active-low LED on GPIO2
led = Pin(2, Pin.OUT, value=1)
my_new_msg = None
TOPIC_BASE = 'malo-iot'

#Control Function
def led_onoff(onoff):
    """ control led ON or OFF
        parameter:
        onoff
            0-->ON, 1-->OFF (acturely, led ON when level=0)
    """
    global led
    
    if(onoff==1):
        led.value(0)
    elif(onoff==-1):
        led.value(not led.value())
    else:
        led.value(1)

def dht_get():
    ''' get dht11 sensor's value (T, H)
        return:
            (Temperature, Humidity)
    '''
    T=None
    H=None
    try:
        dht11 = dht.DHT11(Pin(5)) #D1

        dht11.measure()
        T = dht11.temperature()
        H = dht11.humidity()
    except Exception as e:
        print('dht_get error:', str(e))
    
    return T, H

    
def sub_cb(topic, msg):
    global my_new_msg
    global TOPIC_BASE
    topic_light = TOPIC_BASE+"/light"
    topic_t = TOPIC_BASE+'/T'
    topic_h = TOPIC_BASE+'/H'

    topic = topic.decode('utf-8')
    msg = msg.decode('utf-8')
    my_new_msg = '['+topic+'] '+ msg
    print(my_new_msg)
    
    if(topic == topic_light):
        if msg == "0":
            led_onoff(0)
        else:
            led_onoff(1)
    if(topic == topic_t):
        pass
    if(topic == topic_h):
        pass
    
def main():
    global my_new_msg
    global TOPIC_BASE
    
    mq_fail_count = 0
    tm_pub_th = time.ticks_ms()

    led_onoff(1)

    # Default MQTT server to connect to
    server = "iot.eclipse.org"
    CLIENT_ID = ubinascii.hexlify(machine.unique_id()).decode('utf-8')
    topic_light = TOPIC_BASE+"/light"
    topic_t = TOPIC_BASE+'/T'
    topic_h = TOPIC_BASE+'/H'
    topic_msg = TOPIC_BASE+'/msg'

    c = MQTTClient(CLIENT_ID, server)
    # Subscribed messages will be delivered to this callback
    c.set_callback(sub_cb)
    c.connect()
    c.subscribe(topic_light)
    print("Connected to %s, subscribed to %s topic" % (server, topic_light))

    # wifi ready, blink led
    for i in range(3):
        led_onoff(1)
        time.sleep(1)
        led_onoff(0)
        time.sleep(1)
    print('I am ready!, ID='+str(CLIENT_ID))
    c.publish(topic_msg, 'I am ready!, ID='+str(CLIENT_ID))

    try:
        while 1:            
            try:
                c.check_msg()
                if my_new_msg:
                    c.publish(topic_msg, my_new_msg)
                    my_new_msg = None

                if(time.ticks_ms()-tm_pub_th > 5000):
                    # public some information
                    T, H = dht_get()
                    c.publish(topic_t, str(T))
                    c.publish(topic_h, str(H))
                    
                    tm_pub_th = time.ticks_ms()
                    
            except Exception as e:
                print('ex: ', str(e))
                mq_fail_count+=1
                time.sleep(1)
                
            try:
                if mq_fail_count>5:
                    mq_fail_count=0
                    c = MQTTClient(CLIENT_ID, server)
                    # Subscribed messages will be delivered to this callback
                    c.set_callback(sub_cb)
                    c.connect()
                    c.subscribe(topic_light)
                    print("Connected to %s, subscribed to %s topic" % (server, topic_light))
            except Exception as e:
                print('ex: ', str(e))
                    

            time.sleep(0.001)
                        
    finally:
        c.disconnect()


if __name__ == '__main__':
    main()


Connected to iot.eclipse.org, subscribed to malo-iot/light topic
.I am ready!, ID=a5730800
...[malo-iot/light] 1
[malo-iot/light] 0
.[malo-iot/light] 1
.[malo-iot/light] 0
[malo-iot/light] 1
[malo-iot/light] 0
.[malo-iot/light] 1
..

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 149, in <module>
  File "<stdin>", line 145, in main
  File "<stdin>", line 127, in main
  File "<stdin>", line 111, in main
  File "umqtt/simple.py", line 201, in check_msg
KeyboardInterrupt: 


### 執行的畫面如下：

![linear-mqtt1](image/linear-mqtt3.png)